# Environment

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !pip install transformers
!pip install datasets
!pip install peft
!pip install islab-opendeid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 16.0 MB/s eta 0:00:00


In [3]:
import copy
import io
import math
import os
import time
import re
import json
import random
import numpy as np
import pandas as pd
from datasets import load_dataset, Features, Value, concatenate_datasets, Dataset
from sklearn.model_selection import train_test_split
import torch
from torch.optim import lr_scheduler, AdamW
from torch.utils.data import DataLoader
from torch.nn import functional as F
from tqdm import tqdm, trange
from tqdm.notebook import tqdm
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from transformers import get_linear_schedule_with_warmup
from islab.aicup import OpenDeidBatchSampler
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType, PeftModel

# Constant

In [4]:
"""Basic Setting"""
data_path = "/content/drive/MyDrive/AIcup/data_hpw"
model_path = "/content/drive/MyDrive/AIcup/model_hpw"

"""Data Setting"""
IgnoredPadIdx = -100
PHINullRatio = 0.3
PhiCategory = ['PATIENT', 'DOCTOR', 'USERNAME', 'PROFESSION',
                'ROOM', 'DEPARTMENT', 'HOSPITAL', 'ORGANIZATION',
                'STREET', 'CITY', 'STATE', 'COUNTRY', 'ZIP', 'LOCATION-OTHER',
                'AGE', 'DATE', 'TIME', 'DURATION', 'SET', 'PHONE', 'FAX', 'EMAIL',
                'URL', 'IPADDR', 'SSN', 'MEDICALRECORD', 'HEALTHPLAN', 'ACCOUNT', 'LICENSE',
                'VEHICLE', 'DEVICE', 'BIOID', 'IDNUM']

"""Model Constant Setting"""
BatchSize = 16
Epochs = int(BatchSize*0.75)
MaxLen = 196
LearningRate = 3e-5
WeightDecay = 0.05
PretrainedModel = "EleutherAI/pythia-160m"

# DataReading

In [5]:
"""Filter PHINull"""
def filter_phi_null_ratio(dataset, ratio):
    # get indices
    phi_null_indices = [i for i, item in enumerate(dataset) if item['label'] == 'PHI:Null']
    other_indices = [i for i, item in enumerate(dataset) if item['label'] != 'PHI:Null']
    print(f"Total {len(dataset)}, PHI null {len(phi_null_indices)}, others {len(other_indices)} ")

    # target null count
    target_phi_null_count = int(min(len(phi_null_indices), len(other_indices) * ratio))
    print("target_phi_null_count", target_phi_null_count)

    if len(phi_null_indices) > target_phi_null_count:
        phi_null_indices = random.sample(phi_null_indices, target_phi_null_count)

    # reunion indices
    final_indices = phi_null_indices + other_indices
    random.shuffle(final_indices)

    final_dataset = dataset.select(final_indices)
    return final_dataset

In [6]:
def data_reading(data_path1, data_path2):
  # read data
  data1 = load_dataset("csv", data_files=data_path1, delimiter='\t',
                       features = Features({
                          'fid': Value('string'), 'idx': Value('int64'),
                          'content': Value('string'), 'label': Value('string')}),
                       column_names=['fid', 'idx', 'content', 'label'], keep_default_na=False)["train"]
  data2 = load_dataset("csv", data_files=data_path2, delimiter='\t',
                       features = Features({
                          'fid': Value('string'), 'idx': Value('int64'),
                          'content': Value('string'), 'label': Value('string')}),
                       column_names=['fid', 'idx', 'content', 'label'], keep_default_na=False)["train"]

  print("pahse1 data:", len(data1))
  print("pahse2 data:", len(data2))
  print("data sample:", data1[200])

  # concatenate data
  data = concatenate_datasets([data1, data2])
  print("data length after cancatenation:", len(data))

  # filter data
  filtered_data = filter_phi_null_ratio(data, PHINullRatio)
  # print filter information
  phi_null_count_after = len([item for item in filtered_data if item['label'] == 'PHI:Null'])
  other_count_after = len(filtered_data) - phi_null_count_after
  phi_null_ratio = phi_null_count_after / other_count_after
  print("After filtering:")
  print("Length after filter:", len(filtered_data))
  print("PHI: NULL count:", phi_null_count_after)
  print("Other labels count:", other_count_after)
  print("PHI: NULL to Other labels ratio:", phi_null_ratio)
  return filtered_data

In [7]:
# Original data paths
data_org_train_path1 = os.path.join(data_path, "train_phase1_v8_original_train.tsv")
data_org_train_path2 = os.path.join(data_path, "train_phase2_v8_original_train.tsv")
data_org_test_path1 = os.path.join(data_path, "train_phase1_v8_original_test.tsv")
data_org_test_path2 = os.path.join(data_path, "train_phase2_v8_original_test.tsv")

# Sliced data paths
data_slc_train_path1 = os.path.join(data_path, "train_phase1_v8_sliced_train.tsv")
data_slc_train_path2 = os.path.join(data_path, "train_phase2_v8_sliced_train.tsv")
data_slc_test_path1 = os.path.join(data_path, "train_phase1_v8_sliced_test.tsv")
data_slc_test_path2 = os.path.join(data_path, "train_phase2_v8_sliced_test.tsv")

# Spliced data paths
data_spl_train_path1 = os.path.join(data_path, "train_phase1_v8_spliced_train.tsv")
data_spl_train_path2 = os.path.join(data_path, "train_phase2_v8_spliced_train.tsv")
data_spl_test_path1 = os.path.join(data_path, "train_phase1_v8_spliced_test.tsv")
data_spl_test_path2 = os.path.join(data_path, "train_phase2_v8_spliced_test.tsv")

# Read datasets
data_org_train = data_reading(data_org_train_path1, data_org_train_path2)
data_org_test = data_reading(data_org_test_path1, data_org_test_path2)

data_slc_train = data_reading(data_slc_train_path1, data_slc_train_path2)
data_slc_test = data_reading(data_slc_test_path1, data_slc_test_path2)

data_spl_train = data_reading(data_spl_train_path1, data_spl_train_path2)
data_spl_test = data_reading(data_spl_test_path1, data_spl_test_path2)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

pahse1 data: 48150
pahse2 data: 28840
data sample: {'fid': '106', 'idx': 228, 'content': 'DR CHANG CORIE ROSKE', 'label': 'DOCTOR:CHANG CORIE ROSKE'}
data length after cancatenation: 76990
Total 76990, PHI null 52871, others 24119 
target_phi_null_count 7235
After filtering:
Length after filter: 31354
PHI: NULL count: 7235
Other labels count: 24119
PHI: NULL to Other labels ratio: 0.29997097723786226


...

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

pahse1 data: 3759
pahse2 data: 1576
data sample: {'fid': 'file4000', 'idx': 4449, 'content': '-NEGATIVE FOR MALIGNANCY. B.  LEFT GROIN NODES &amp;#8211; -NEGATIVE FOR MALIGNANCY. C.  VULVA &amp;#8211; ', 'label': 'PHI:Null'}
data length after cancatenation: 5335
Total 5335, PHI null 2802, others 2533 
target_phi_null_count 759
After filtering:
Length after filter: 3292
PHI: NULL count: 759
Other labels count: 2533
PHI: NULL to Other labels ratio: 0.2996446900908014


In [8]:
train_data = concatenate_datasets([data_org_train, data_slc_train, data_spl_train])
print("total train data:", len(train_data))
train_data = train_data.shuffle(seed=42)
# train_data = train_data.select(range(20000))
# print("sampled train data:", len(train_data))
for i in range(100, 103):
  print("train data samples:", train_data[i])

test_data = concatenate_datasets([data_org_test, data_slc_test, data_spl_test])
test_data = test_data.shuffle(seed=42)
print("total test data:", len(test_data))
test_data = test_data.select(range(3000))
print("sampled test data:", len(test_data))
for i in range(100, 103):
  print("test data samples:", test_data[i])

total train data: 70116
train data samples: {'fid': '527', 'idx': 49, 'content': 'Last edited : 10/1/2063  Page: 2', 'label': 'DATE:10/1/2063'}
train data samples: {'fid': '102', 'idx': 50, 'content': 'Lab No:  62T18872,62T18872 E Onwentsia KURRAJONG HEIGHTS  Victoria  0886 Specimen: Fluid,Tissue D.O.B:  3/2/2024 Sex:  F ', 'label': 'IDNUM:62T18872++IDNUM:62T18872++STREET:E Onwentsia++CITY:KURRAJONG HEIGHTS++STATE:Victoria++ZIP:0886++DATE:3/2/2024'}
train data samples: {'fid': '165', 'idx': 7099, 'content': 'B. Uterus, cervix, left fallopian tube and ovary', 'label': 'PHI:Null'}
total test data: 6681
sampled test data: 3000
test data samples: {'fid': 'file1861', 'idx': 19, 'content': 'MRN no: 1392651', 'label': 'MEDICALRECORD:1392651'}
test data samples: {'fid': '149', 'idx': 60, 'content': 'Lab No:  57O05598 E Evergreen BUCASIA  Tasmania  6434 Specimen: Tissue D.O.B:  16/12/1990 Sex:  F Collected: 18/2/2063 at : ', 'label': 'IDNUM:57O05598++STREET:E Evergreen++CITY:BUCASIA++STATE:Tasm

# Model

In [9]:
bos = '<|endoftext|>'
eos = '<|END|>'
pad = '<|pad|>'
sep ='\n\n####\n\n'

special_tokens_dict = {'eos_token': eos, 'bos_token': bos, 'pad_token': pad, 'sep_token': sep}

tokenizer = AutoTokenizer.from_pretrained(PretrainedModel, revision="step3000")
tokenizer.padding_side = 'left'
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print(f"{tokenizer.pad_token}: {tokenizer.pad_token_id}")

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

<|pad|>: 50278


In [10]:
"""model config"""
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config = AutoConfig.from_pretrained(PretrainedModel,
                                    bos_token_id=tokenizer.bos_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                    pad_token_id=tokenizer.pad_token_id,
                                    sep_token_id=tokenizer.sep_token_id,
                                    output_hidden_states=False,
                                    attention_dropout=0.3)

model = AutoModelForCausalLM.from_pretrained(PretrainedModel, revision="step3000", config=config)
model.resize_token_embeddings(len(tokenizer))

"""LoRA"""
lora_config = LoraConfig(
 r=16,
 lora_alpha=8,
 target_modules=['query_key_value', 'dense'],
 lora_dropout=0.7,
 bias="none",
 task_type="CAUSAL_LM"
)


model = prepare_model_for_int8_training(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

model.to(device)

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/375M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:136: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


trainable params: 884,736 || all params: 163,170,816 || trainable%: 0.5422146077886869


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(50280, 768)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-11): 12 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): Linear(
                in_features=768, out_features=2304, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.7, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_

In [11]:
# optimizer = AdamW(model.parameters(), lr=LearningRate, weight_decay=WeightDecay)
# Filter the parameters you want to train
trainable_params = [p for p in model.parameters() if p.requires_grad]
# Set up the optimizer to optimize only these parameters
optimizer = AdamW(trainable_params, lr=LearningRate, weight_decay=WeightDecay)
num_training_steps = len(train_data) // BatchSize * Epochs
CountSteps = int(num_training_steps*0.1/Epochs) // 10 *10
if CountSteps == 0:
  CountSteps = 10
WarmUpSteps = int(num_training_steps*0.01)
print(f"training steps {num_training_steps}, count each {CountSteps} steps, warm up in {WarmUpSteps} ")
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=WarmUpSteps,
    num_training_steps=num_training_steps
)

training steps 52584, count each 430 steps, warm up in 525 


# Train Dataloader

In [12]:
def collate_batch_with_prompt_template(batch, tokenizer, template = "<|endoftext|> __CONTENT__\n\n####\n\n__LABEL__ <|END|>", IGNORED_PAD_IDX = -100):
    """ template: __CONTENT__ and __LABEL__ will be replaced with the content and the corresponding labels."""
    # default template: {bos} {data['content']} {sep}

    texts = [template.replace("__LABEL__", data['label']).replace("__CONTENT__", data['content']) for data in list(batch)]
    # encoded_seq = tokenizer(texts, padding=True, trancution=True, max_len = MaxLen)
    encoded_seq = tokenizer(texts, padding=True, truncation=True, max_length=MaxLen)

    indexed_tks = torch.tensor(encoded_seq['input_ids'])
    attention_mask = torch.tensor(encoded_seq['attention_mask'])
    encoded_label = torch.tensor(encoded_seq['input_ids'])
    encoded_label[encoded_label == tokenizer.pad_token_id] = IGNORED_PAD_IDX

    return indexed_tks, encoded_label, attention_mask

In [13]:
train_data = list(train_data)
test_data = list(test_data)

train_dataloader = DataLoader(train_data,
                              batch_sampler=OpenDeidBatchSampler(train_data, BatchSize),
                              collate_fn=lambda batch: collate_batch_with_prompt_template(batch, tokenizer),
                              pin_memory=True)

test_dataloader = DataLoader(test_data,
                             batch_sampler=OpenDeidBatchSampler(test_data, 1),
                             collate_fn=lambda batch: collate_batch_with_prompt_template(batch, tokenizer),
                             pin_memory=True)

dataloaders = {"train": train_dataloader, "test": test_dataloader}

# Training

In [14]:
def write_losses(loss_path, train_losses, dev_losses):
  losses={}
  losses["train"] = train_losses
  losses["test"] = dev_losses
  with open(loss_path, "w") as out_config:
    json.dump(losses, out_config, indent=4)

In [ ]:
"""Train Model"""
# model path
name = str(int(time.time()))
save_path = os.path.join(model_path, name)
model_name = save_path + f"/best_{name}.pt"

loss_path = os.path.join(save_path, "loss.csv")

if not os.path.isdir(save_path):
    os.mkdir(save_path)

best_loss = float('inf')
train_losses = []
test_losses = []

for epoch in range(Epochs):
    print("[Training] Epoch {}/{}".format(epoch, Epochs - 1))
    print("-" * 10)

    running_loss_train = 0.0
    running_loss_test = 0.0
    intermediate_loss = 0.0

    # Training phase
    model.train()
    for count, (seqs, labels, masks) in enumerate(tqdm(dataloaders["train"])):
        seqs, labels, masks = seqs.to(device), labels.to(device), masks.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=seqs, labels=labels, attention_mask=masks)
        # loss = outputs.loss.mean()
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        scheduler.step()

        running_loss_train += loss.item()
        intermediate_loss += loss.item()
        # Print training losses
        if count % CountSteps == 0 and count != 0:
            average_loss = intermediate_loss / CountSteps
            print(f"Loss in epoch{epoch}-step{count}: {average_loss:.4f}")
            train_losses.append(average_loss)
            intermediate_loss = 0

    # Print epoch train losses
    epoch_train_loss = running_loss_train / len(dataloaders["train"])
    print(f"[Training] Train Loss: {epoch_train_loss:.4f}")

    # Testing phase
    model.eval()
    with torch.no_grad():
        for seqs_test, labels_test, masks_test in tqdm(dataloaders["test"]):
            seqs_test, labels_test, masks_test = seqs_test.to(device), labels_test.to(device), masks_test.to(device)
            outputs = model(input_ids=seqs_test, labels=labels_test, attention_mask=masks_test)
            # loss = outputs.loss.mean()
            loss = outputs.loss
            running_loss_test += loss.item()

    # print epoch test loss
    epoch_test_loss = running_loss_test / len(dataloaders["test"])
    print(f"[Training] Test Loss: {epoch_test_loss:.4f}")

    # save the best model
    test_losses.append(epoch_test_loss)

    if epoch_test_loss < best_loss:
        # write losses in each epoch
        write_losses(loss_path, train_losses, test_losses)
        best_loss = epoch_test_loss
        best_model_wts = copy.deepcopy(model.state_dict())
        # torch.save(model.state_dict(), model_name)
        model.save_pretrained(model_name)
        print(f"[INFO] Updated best model on dev checkpoint: {model_name}")

[Training] Epoch 0/11
----------


  0%|          | 0/4383 [00:00<?, ?it/s]

Loss in epoch0-step430: 6.1741
Loss in epoch0-step860: 4.9155
Loss in epoch0-step1290: 4.2746
Loss in epoch0-step1720: 4.0993
Loss in epoch0-step2150: 5.0738
Loss in epoch0-step2580: 5.7336


# Val Dataloader

In [ ]:
val_path = os.path.join(data_path, "valid_phase1_128_repeat.tsv")
data_val = load_dataset("csv", data_files=val_path, delimiter='\t',
                          features = Features({
                              'fid': Value('string'), 'idx': Value('int64'),
                              'content': Value('string'), 'label': Value('string')}),
                              column_names=['fid', 'idx', 'content', 'label'])
data_val= list(data_val['train'])
# data_val = data_val[:1000]
print("pahse1 validation:", len(data_val))
print("validation sample:", data_val[1])

# Generation

In [ ]:
name = "1700311585"
time = name[:10]
model_name = f"drive/MyDrive/AIcup/model_hpw/{name}/best_{time}.pt"
answer_path = f"drive/MyDrive/AIcup/model_hpw/{name}/answer_{time}.txt"
prediction_path = f"drive/MyDrive/AIcup/model_hpw/{name}/prediction_{time}.txt"
model.load_state_dict(torch.load(model_name))
model = model.to(device)

In [ ]:
def aicup_predict(model, tokenizer, input, template = "<|endoftext|> __CONTENT__\n\n####\n\n"):
    seeds = [template.replace("__CONTENT__", data['content']) for data in input]
    sep = tokenizer.sep_token
    eos = tokenizer.eos_token
    pad = tokenizer.pad_token
    pad_idx = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
    """Generate text from a trained model."""
    model.eval()
    device = model.device
    texts = tokenizer(seeds, return_tensors = 'pt', padding=True).to(device)
    predictions = []

    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**texts, max_new_tokens=400, pad_token_id = pad_idx,
                                        eos_token_id=tokenizer.convert_tokens_to_ids(eos))
        preds = tokenizer.batch_decode(output_tokens)
        for idx , pred in enumerate(preds):
          if "NULL" in pred:
            continue
          phi_infos = pred[pred.index(sep)+len(sep):].replace(pad, "").replace(eos, "").strip()
          predictions.append(f'{input[idx]["fid"]}\t{input[idx]["idx"]}\t{input[idx]["content"]}\t{phi_infos}')
    return predictions

In [ ]:
ValBatchSize = 32
with open(os.path.join(prediction_path), 'w', encoding='utf8') as f_predictions:
    for i in tqdm(range(0, len(data_val), ValBatchSize)):
        with torch.no_grad():
            seeds = data_val[i:i+ValBatchSize]
            predictions = aicup_predict(model, tokenizer, input=seeds)
            for p in predictions:
                f_predictions.write(p + '\n')